<a href="https://colab.research.google.com/github/amorfinv/icrat_notebooks/blob/main/geofences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding geofences to BlueSky

---


The goal of this notebook is to show how to use data from OpenStreetMap to create geofenced areas. The example will look at an area in Tampa find all of the parks and save them as a ```geojson``` file so that it can be imported into BlueSky.

Start by importing ```osmnx```, ```geopandas```, ```shapely```. Note that ```leafmap``` is only neeeded for visualizing maps in this notebook.

In [1]:
import subprocess

try:
    import osmnx as ox
except ImportError:
    print('Installing osmnx ...')
    # pip installing osmnx brings in geopandas, shapely, and numpy
    subprocess.check_call(["python", '-m', 'pip', 'install', 'osmnx'])
    import osmnx as ox

# Leafmap is only necessary for visualizing maps in the notebook
try:
    import leafmap
except ImportError:
    print('Installing leafmap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'leafmap'])
    import leafmap

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiLineString
from shapely import ops

Installing osmnx ...
Installing leafmap ...


## Choose an area of interest: Downtown Tampa

An area of Downtown Tampa with a couple of parks is chosen as our area of interest.

In [2]:
# Show map of Tampa
map_args = {"center":(27.94650, -82.46071), "zoom":16, "height":"500px", "width":"500px"}
cartomap = {
    "url":"https://cartodb-basemaps-b.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",
    "name":"CartoDB",
    "attribution":"CartoDB",
}
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
display(m)

Map(center=[27.9465, -82.46071], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Get the parks with ```osmnx```

All parks in the area of interests will be set as geofences in BlueSky.




In [3]:
# Create a polygon of surrounding area

p1 = Point(-82.46491, 27.94375)
p2 = Point(-82.46504, 27.95078)
p3 = Point(-82.45727, 27.95088)
p4 = Point(-82.45706, 27.944171)

polygon = Polygon([p1, p2, p3, p4, p1])

# Get geometries from polygon
gdf_parks = ox.geometries_from_polygon(polygon, tags={'leisure': 'park'})

# Create gdf for map
gdf_area = gpd.GeoDataFrame(geometry=[polygon], crs='epsg:4326')

# create map and add gdf
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_gdf(gdf_area, layer_name='area', style={"weight": 3}, zoom_to_layer=False)
m.add_gdf(gdf_parks, layer_name='park', style={"weight": 3}, zoom_to_layer=False)
display(m)

/usr/local/lib/python3.7/dist-packages/ipywidgets/widgets/widget.py:477: DeprecationWarning: Passing unrecognized arguments to super(Map).__init__(height='500px', width='500px').
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Map(center=[27.9465, -82.46071], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

# Prepare the ```GeoDataFrame``` for BlueSky

In [4]:
# Drop all columns except geometry
gdf_parks.drop(gdf_parks.iloc[:, 1:],axis = 1, inplace=True)

# create a column for the bottom and top heights of the geofence. 
# Here we will set the bottom to 0 and top to 300 feet.
gdf_parks['bottom'] = 0
gdf_parks['top'] = 300

# use the osmid as the name
gdf_parks['name'] = [osmid for _, osmid in gdf_parks.index]

# reset index
gdf_parks.reset_index(inplace=True, drop=True)

# save as GeoJson to import to BlueSky
gdf_parks.to_file('parks.geojson', driver='GeoJSON')